# VOBHS Indicator: 

## Algorithmic Trading Indicator Analysis & Backtest

Use the "Run" button to execute the code.

## Importing packages:

In [1]:
!pip install backtesting dateparser ccxt pyti jovian --upgrade --quiet

In [2]:
import ccxt
import jovian
import requests
import math
import datetime
import concurrent
import os
import glob
import time
import dateparser

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None  # default='warn'

from pyti.hull_moving_average import hull_moving_average as hull_moving_average
from pyti.average_true_range import average_true_range as average_true_range

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import GOOG

/opt/conda/lib/python3.9/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

## Importing Data:

### Historical data from [cryptodatadownload.com](https://www.cryptodatadownload.com/data/binance/):

- I personally would avoid this choice due to the quality of the dataset
- However, this is added here for reading your personal datasets

### Historical 1 hr data from [CCXT Library](https://ccxt.com/)


In [3]:
binance = ccxt.binanceusdm({'options': {'enableRateLimit': True}})
symbol = 'ETHUSDT'
interval = '5m'

data = pd.DataFrame(binance.fetch_ohlcv( symbol, interval, limit=1500))
data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
data['Date'] = pd.to_datetime(data['Date'], unit='ms') + pd.Timedelta(hours=2)
data['Symbol'] = symbol
data = data.set_index('Date')

In [4]:
atr_multiplier = 2.5
data['atr'] = average_true_range(data['Close'],3)
data['sell_stop_loss'] = (data['Close'] + data['atr'] * atr_multiplier) #atr_upper_band
data['buy_stop_loss'] = (data['Close'] - data['atr'] * atr_multiplier) #atr_lower_band
data['buy_stop_loss'] = data['buy_stop_loss'].round(decimals=2)
data['sell_stop_loss'] = data['sell_stop_loss'].round(decimals=2)
data.tail()

,Open,High,Low,Close,Volume,Symbol,atr,sell_stop_loss,buy_stop_loss
Date,,,,,,,,,
2022-08-07 19:50:00,1711.12,1713.36,1709.00,1711.15,17494.320,ETHUSDT,1.600499,1715.15,1707.15
2022-08-07 19:55:00,1711.15,1712.32,1710.37,1710.86,5546.360,ETHUSDT,1.163666,1713.77,1707.95
2022-08-07 20:00:00,1710.86,1714.00,1710.38,1710.55,7840.214,ETHUSDT,0.975777,1712.99,1708.11
2022-08-07 20:05:00,1710.56,1710.70,1708.63,1708.71,7035.233,ETHUSDT,1.367185,1712.13,1705.29
2022-08-07 20:10:00,1708.71,1710.18,1707.56,1708.50,10734.590,ETHUSDT,1.594790,1712.49,1704.51


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1500 entries, 2022-08-02 15:15:00 to 2022-08-07 20:10:00
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Open            1500 non-null   float64
 1   High            1500 non-null   float64
 2   Low             1500 non-null   float64
 3   Close           1500 non-null   float64
 4   Volume          1500 non-null   float64
 5   Symbol          1500 non-null   object 
 6   atr             1498 non-null   float64
 7   sell_stop_loss  1498 non-null   float64
 8   buy_stop_loss   1498 non-null   float64
dtypes: float64(8), object(1)
memory usage: 117.2+ KB


## Setting up Indicators:

In [6]:
def ATR(data):
    atr_multiplier = 2.5
    data['atr'] = average_true_range(data['Close'],3)
    data['sell_stop_loss'] = (data['Close'] + data['atr'] * atr_multiplier) #atr_upper_band
    data['buy_stop_loss'] = (data['Close'] - data['atr'] * atr_multiplier) #atr_lower_band
    data['buy_stop_loss'] = data['buy_stop_loss'].round(decimals=2)
    data['sell_stop_loss'] = data['sell_stop_loss'].round(decimals=2)
    return data['buy_stop_loss'], data['sell_stop_loss']

def TP(data):
    data['buy_TP'] = (data['Open'] - data['buy_stop_loss'].shift(1))/0.4 + data['sell_stop_loss'] 
    data['sell_TP'] = (data['Open'] - data['sell_stop_loss'].shift(1))/0.4 + data['sell_stop_loss']
    return data['buy_TP'],data['sell_TP']
        
def VO(data, Open, Close): 
    length = 100

    data['spike'] = Close - Open
    data['x'] = data['spike'].rolling(length).std()
    data['y'] = data['x'] * -1

    data['above'] = data['spike'] > data['x']
    data['below'] = data['spike'] < data['y']

    data['vo_status'] =  data.apply(lambda x: 1 if x.above == True 
                               else (-1 if x.below == True 
                               else 0), axis =1)

    data = data.drop(['above','below'], axis=1)
    data['vo_status'] =data['vo_status'].shift(-1)
    return data['vo_status']

def HULL_color(data, Close):
    #Hull MA
    data['m_hull'] = hull_moving_average(Close, 400) #length premultiplied by 2
    data['s_hull'] = data['m_hull'].shift(2)
    data['m_s_above'] = data['m_hull'] > data['s_hull']
    data['hull_color_status'] = data.apply(lambda x: 1 if x.m_s_above == True else -1, axis =1)

    return data['hull_color_status']

def HULL_price(data, Close):

    #Hull MA
    data['m_hull'] = hull_moving_average(Close, 400) #length premultiplied by 2
    data['s_hull'] = data['m_hull'].shift(2)
    data['m_s_above'] = data['m_hull'] > data['s_hull']
    data['hull_color_status'] = data.apply(lambda x: 1 if x.m_s_above == True else -1, axis =1)

    return data['hull_color_status']

def high_pass_filter(Close):
    PI = np.pi
    HP = []

    for i, _ in enumerate(Close):
        if i < 2:
            HP.append(0)
        else:
            angle = 0.707 * 2 * PI/100
            alpha1 = (math.cos(angle) + math.sin(angle)-1)/ math.cos(angle)
            HP.append(math.pow(1.0-alpha1/2.0, 2)*(Close[i]-2*Close[i-1]+Close[i-2]) + 2*(1-alpha1)*HP[i-1] - math.pow(1-alpha1, 2)*HP[i-2])

    return HP

def super_smoother(HP, LPPeriod):
    Filt = []
    for i, _ in enumerate(HP):
        if i < 2:
            Filt.append(0)
        else:
            arg = 1.414 * 3.14159 / LPPeriod
            a_1 = math.exp(-arg)
            b_1 = 2 * a_1 * math.cos(4.44/float(LPPeriod))
            c_2 = b_1
            c_3 = -a_1 * a_1
            c_1 = 1 - c_2 - c_3
            Filt.append(c_1 * (HP[i] + HP[i-1]) / 2 + c_2 * Filt[i-1] + c_3 * Filt[i-2])
    return Filt

def agc(Filt):
    X = []
    Peak = []
    for i, _ in enumerate(Filt):
        if i < 1:
            X.append(0)
            Peak.append(0)
            #Peak.append(.0000001)
        else:
            Peak.append(0.991 * Peak[i - 1])
            if abs(Filt[i]) > Peak[i]:
                Peak[i] = abs(Filt[i])

            if Peak[i] != 0:
                X.append(Filt[i] / Peak[i])

    return X

def quotient(X, K_val):

    K = K_val
    Q = []
    for i, _ in enumerate(X):
        if i<1:
            Q.append(0)
        else:
            Q.append((X[i]+ K) / (K*X[i]+1))
    return Q

#Input Quotient to return smoothen the quotient
def sma(Q1, length):
    trigger = []
    for i, _ in reversed(list(enumerate(Q1))):
        sum = 0
        for t in range(i - length + 1, i + 1):
            sum = sum + Q1[t] / length
        trigger.insert(0, sum)
    return trigger

def EOT(data, Close):
    K1 = 0
    K2 = 0.4
    LPPeriod_1 = 6
    LPPeriod_2 = 27

    HP = high_pass_filter(data.Close)

    #quotient 1 params
    Filt_1 = super_smoother(HP, LPPeriod_1)
    X_1 = agc(Filt_1)

    #quotient 2 params
    Filt_2 = super_smoother(HP, LPPeriod_2)
    X_2 = agc(Filt_2)

    data = data.reset_index()
    q1 = quotient(X_1, K1)
    q2 = quotient(X_2, K2)

    trig = sma(q1,length=2)
    data['trig'] = pd.Series(trig)

    data['white_line'] = data['trig']
    data['red_line'] = pd.Series(q2)

    data['white_line'] = (data['white_line']*100)+110
    data['red_line'] = (data['red_line']*100)+110
    
    
    data['prev_white_line'] = data['white_line']
    data['prev_white_line'] = data['prev_white_line'].shift(1)

    data['prev_red_line'] = data['red_line']
    data['prev_red_line'] = data['prev_red_line'].shift(1)

    #data = data.fillna(method='ffill')
    #data.dropna(inplace=True)

    data['bullish_cross'] = (data['prev_white_line'] < data['red_line']) & (data['white_line'] > data['red_line']) & (data['white_line']<85)

    data['bearish_cross'] = (data['prev_red_line'] < data['white_line']) & (data['red_line'] > data['white_line']) & (data['red_line']>15)
    data['crossover'] = data.apply(lambda x: 'bullish crossover' if x.bullish_cross == True
                          else ('bearish crossover' if x.bearish_cross == True else 'none'), axis=1)
    data = data.drop(['bullish_cross','bearish_cross'], axis=1)

    data = data[data.columns.drop(list(data.filter(regex='index')))]


    data['buy'] = (data['crossover']=='bullish crossover') #& (data['hull_color_status']=='green') & (data['hull_price_status']=='above') & (data['vo_status']=='above')
    data['sell'] = (data['crossover']=='bearish crossover') #& (data['hull_color_status']=='red') & (data['hull_price_status']=='below') & (data['vo_status']=='below')

    data['signal'] = data.apply(lambda x: 1 if x.buy == True
                              else (-1 if x.sell == True else 0), axis=1)


    return data['white_line'], data['red_line'], data['signal']


In [7]:
K1 = 0
K2 = 0.4
LPPeriod_1 = 6
LPPeriod_2 = 27

HP = high_pass_filter(data.Close)

#quotient 1 params
Filt_1 = super_smoother(HP, LPPeriod_1)
X_1 = agc(Filt_1)

#quotient 2 params
Filt_2 = super_smoother(HP, LPPeriod_2)
X_2 = agc(Filt_2)

data = data.reset_index()
q1 = quotient(X_1, K1)
q2 = quotient(X_2, K2)

trig = sma(q1,length=2)
data['trig'] = pd.Series(trig)

data['white_line'] = data['trig']
data['red_line'] = pd.Series(q2)

data['white_line'] = (data['white_line']*100)+110
data['red_line'] = (data['red_line']*100)+110


data['prev_white_line'] = data['white_line']
data['prev_white_line'] = data['prev_white_line'].shift(1)

data['prev_red_line'] = data['red_line']
data['prev_red_line'] = data['prev_red_line'].shift(1)

#data = data.fillna(method='ffill')
#data.dropna(inplace=True)

data['bullish_cross'] = (data['prev_white_line'] < data['red_line']) & (data['white_line'] > data['red_line']) & (data['white_line']<85)

data['bearish_cross'] = (data['prev_red_line'] < data['white_line']) & (data['red_line'] > data['white_line']) & (data['red_line']>15)
data['crossover'] = data.apply(lambda x: 'bullish crossover' if x.bullish_cross == True
                      else ('bearish crossover' if x.bearish_cross == True else 'none'), axis=1)
data = data.drop(['bullish_cross','bearish_cross'], axis=1)

data = data[data.columns.drop(list(data.filter(regex='index')))]


data['buy'] = (data['crossover']=='bullish crossover') #& (data['hull_color_status']=='green') & (data['hull_price_status']=='above') & (data['vo_status']=='above')
data['sell'] = (data['crossover']=='bearish crossover') #& (data['hull_color_status']=='red') & (data['hull_price_status']=='below') & (data['vo_status']=='below')

data['signal'] = data.apply(lambda x: 1 if x.buy == True
                          else (-1 if x.sell == True else 0), axis=1)


In [14]:
buy_stop_loss, sell_stop_loss = ATR(data)
buy_TP, sell_TP = TP(data)
white_line, red_line, signal = EOT(data, data.Close)
vo_status = VO(data, data.Open, data.Close)
hull_color = HULL_color(data, data.Close)
hull_price = HULL_price(data, data.Close)

In [9]:
data[(data['signal']==1) & (data['hull_color_status'] ==1) & (data['vo_status']==1)]

,Date,Open,High,Low,Close,Volume,Symbol,atr,sell_stop_loss,buy_stop_loss,...,spike,x,y,above,below,vo_status,m_hull,s_hull,m_s_above,hull_color_status
774,2022-08-05 07:45:00,1651.22,1657.41,1651.21,1657.28,23337.298,ETHUSDT,3.992822,1667.26,1647.30,...,6.06,3.284786,-3.284786,True,False,1,1623.918058,1622.687338,True,1
816,2022-08-05 11:15:00,1655.90,1666.13,1655.35,1660.72,56838.024,ETHUSDT,2.962272,1668.13,1653.31,...,4.82,3.558354,-3.558354,True,False,1,1651.357832,1650.338635,True,1
1474,2022-08-07 18:05:00,1695.66,1698.34,1695.03,1698.34,10473.712,ETHUSDT,2.547111,1704.71,1691.97,...,2.68,2.565040,-2.565040,True,False,1,1685.462189,1684.963066,True,1


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               1500 non-null   datetime64[ns]
 1   Open               1500 non-null   float64       
 2   High               1500 non-null   float64       
 3   Low                1500 non-null   float64       
 4   Close              1500 non-null   float64       
 5   Volume             1500 non-null   float64       
 6   Symbol             1500 non-null   object        
 7   atr                1498 non-null   float64       
 8   sell_stop_loss     1498 non-null   float64       
 9   buy_stop_loss      1498 non-null   float64       
 10  trig               1499 non-null   float64       
 11  white_line         1499 non-null   float64       
 12  red_line           1499 non-null   float64       
 13  prev_white_line    1499 non-null   float64       
 14  prev_red

## Backtesting

In [16]:
class Vobhs_Strategy(Strategy):
    def init(self):
        
        self.buy_stop_loss, self.sell_stop_loss = self.I(ATR,data)
        self.buy_TP, self.sell_TP = self.I(TP,data)
        self.white_line, self.red_line, self.signal = self.I(EOT, data, data.Close)
        self.vo_status = self.I(VO, data, data.Open, data.Close)
        self.hull_color = self.I(HULL_color, data, data.Close)
        self.hull_price = self.I(HULL_price, data, data.Close)
        
        
    def next(self):
        price = self.data.Close[-1]
        
        if (self.hull_color == 1 and
            self.vo_status == 1 and 
            self.signal==1):
            
            self.buy(sl= self.buy_stop_loss, tp= 1.4*self.buy_TP)
        elif (self.hull_color == -1 and
            self.vo_status == -1 and 
            self.signal== -1):
            self.sell(sl= 0.97*self.sell_stop_loss, tp= self.sell_TP)

In [37]:
class Vobhs_Strategy(Strategy):
    def init(self):
        self.buy_stop_loss, self.sell_stop_loss = self.I(ATR,data)
        self.buy_TP, self.sell_TP = self.I(TP,data)
        self.white_line, self.red_line, self.signal = self.I(EOT, data, data.Close)
        self.vo_status = self.I(VO, data, data.Open, data.Close)
        self.hull_color = self.I(HULL_color, data, data.Close)
        self.hull_price = self.I(HULL_price, data, data.Close)
        
        
    def next(self):
        
        price = self.data.Close[-1]
        
        if (self.hull_color == 1 and
             self.vo_status == 1 and
             self.signal==1):
            
            self.buy(sl= self.buy_stop_loss, tp= 1.5*self.buy_TP)
        elif (self.hull_color == -1 and
              self.vo_status == -1 and 
              self.signal== -1):
            
            self.sell(sl= self.sell_stop_loss, tp= 0.97*self.sell_TP)

In [38]:
backtest = Backtest(data, Vobhs_Strategy, commission=0.02)
backtest.run()

/tmp/ipykernel_39/3071365595.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  backtest = Backtest(data, Vobhs_Strategy, commission=0.02)


Start                                     0.0
End                                    1499.0
Duration                               1499.0
Exposure Time [%]                        45.6
Equity Final [$]                     10098.46
Equity Peak [$]                      10289.16
Return [%]                             0.9846
Buy & Hold Return [%]                8.372291
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -3.762212
Avg. Drawdown [%]                   -1.239755
Max. Drawdown Duration                  500.0
Avg. Drawdown Duration                 84.875
# Trades                                  1.0
Win Rate [%]                            100.0
Best Trade [%]                       1.165885
Worst Trade [%]                      1.165885
Avg. Trade [%]                    

In [39]:
backtest.plot()

Row(id='1772', ...)

In [ ]:
# Execute this to save new versions of the notebook
jovian.commit(project="vobhs-backtest")

<IPython.core.display.Javascript object>